In [1]:
!pip install pillow
import numpy as np
from moviepy.editor import VideoFileClip

In [2]:
def lighting_change(frame1, frame2):
    if len(frame1.shape) == 3:
        frame1 = np.dot(frame1[..., :3], [0.2989, 0.5870, 0.1140])
    if len(frame2.shape) == 3:
        frame2 = np.dot(frame2[..., :3], [0.2989, 0.5870, 0.1140])
    hist1, _ = np.histogram(frame1, bins=256, range=(0, 256))
    hist2, _ = np.histogram(frame2, bins=256, range=(0, 256))

    hist_diff = np.sum(np.abs(hist1 - hist2))
    
    return hist_diff


def blur(frame):
    grayscale_frame = np.dot(frame[..., :3], [0.2989, 0.5870, 0.1140])

    gradient = np.gradient(grayscale_frame)

    gradient_variance = np.var(gradient)

    return gradient_variance


def blocking_change(frame1, frame2):
    grayscale_frame1 = np.dot(frame1[..., :3], [0.2989, 0.5870, 0.1140])
    grayscale_frame2 = np.dot(frame2[..., :3], [0.2989, 0.5870, 0.1140])

    diff = np.abs(grayscale_frame1 - grayscale_frame2)

    threshold = 50
    return np.sum(diff > threshold)


def camera_movement(frame1, frame2):
    grayscale_frame1 = np.dot(frame1[..., :3], [0.2989, 0.5870, 0.1140])
    grayscale_frame2 = np.dot(frame2[..., :3], [0.2989, 0.5870, 0.1140])

    diff = np.abs(grayscale_frame1 - grayscale_frame2)

    threshold = 50

    num_changed_pixels = np.sum(diff > threshold)
    total_pixels = np.prod(frame1.shape[:-1])
    percent_changed = (num_changed_pixels / total_pixels) * 100

    return percent_changed


def calculate_bitrate(file_size, duration):
    return file_size / duration


def extract_features(frame1, frame2, file_size, duration):
    return {
        'bitrate': calculate_bitrate(file_size, duration),
        'lighting_change': lighting_change(frame1, frame2),
        'blur': blur(frame2),
        'blocking_change': blocking_change(frame1, frame2),
        'camera_movement': camera_movement(frame1, frame2)
    }

def calculate_file_info(video_files):
    file_info = {}
    for file in video_files:
        clip = VideoFileClip(file)
        file_size = clip.reader.size
        duration = clip.duration
        file_info[file] = {'file_size': file_size, 'duration': duration}
    return file_info

In [11]:
def process_frames_with_anomaly(frames, anomaly_timecodes, file_size, duration):
    features = {'Time': [],
                    'Frame': [],
                    'Lighting_Changes': [],
                    'Blurriness': [],
                    'Blocking_Camera': [],
                    'Camera_Movement': [],
                    'File_Size': [],
                    'Duration':[]}
    prev_frame = None
    prev_features = None

    current_anomaly = None
    next_anomaly_time = None

    for frame_num, frame in enumerate(frames):
        # проверка, если фрейм в аномалии
        anomaly_period = False
        for timecode, periods in anomaly_timecodes.items():
            for anomaly_type, anomaly_periods in periods.items():
                for period in anomaly_periods:
                    start_time, end_time = period  # вытаскиваем начало и конец аномального периода
                    if frame_num >= start_time and frame_num <= end_time:
                        anomaly_period = True
                        current_anomaly = timecode
                        next_anomaly_time = end_time

        if not anomaly_period:
            # если не в аномалии, то первый раз все считаем
            if prev_features is None:
                prev_features = {
                    'Lighting_Changes': lighting_change(prev_frame, frame),
                    'Blurriness': blur(frame),
                    'Blocking_Camera': blocking_change(prev_frame, frame),
                    'Camera_Movement': camera_movement(prev_frame, frame),
                    'File_Size': file_size,
                    'Duration': duration
                }
            # дополняем до след периода
            while frame_num < next_anomaly_time:
                features['Time'].append(current_anomaly)
                features['Frame'].append(frame_num)
                for name, val in prev_features.items():
                    features[name].append(val)
                frame_num += 1

        else:
            # если в аномалии то просто все считаем
            if prev_frame is not None:
                lighting_changes_am = lighting_change(prev_frame, frame)
                blurriness = blur(frame)
                blocking_camera = blocking_change(prev_frame, frame)
                camera_move = camera_movement(prev_frame, frame)

                little_map={
                    'Time': current_anomaly,
                    'Frame': frame_num,
                    'Lighting_Changes': lighting_changes_am,
                    'Blurriness': blurriness,
                    'Blocking_Camera': blocking_camera,
                    'Camera_Movement': camera_move,
                    'File_Size': file_size,
                    'Duration': duration
                }
                for name, arr in features.items():
                    arr.append(little_map[name])
                
        prev_frame = frame

    return features


In [4]:
import glob
train_path="../Train/anomaly/*.mp4"
files=glob.glob(train_path)
files

['../Train/anomaly\\0.mp4',
 '../Train/anomaly\\2.mp4',
 '../Train/anomaly\\3.mp4',
 '../Train/anomaly\\4.mp4',
 '../Train/anomaly\\5.mp4']

In [5]:
files=['../Train/anomaly\\3.mp4',
 '../Train/anomaly\\4.mp4',
 '../Train/anomaly\\5.mp4']

In [12]:
from threading import Thread
anomaly_data = {
    '3': {
        'Blur': [(19, 59), (79, 138)],
        'Motion': [(0, 19), (40, 60), (80, 106), (121, 141)]
    },
    '4': {
        'Overlay': [(17, 28), (55, 64), (174, 185)],
        'Overexposure': [(28, 30), (64, 70)],
        'Blur': [(83, 153)]
    },
    '5': {
        'Overlay': [(662, 675), (679, 690)],
        'Motion': [(3, 4), (324, 325), (454, 455), (507, 508), (508, 560), (547, 662), (714, 718)],
        'Blur': [(161, 221)]
    }
  }



file_info = calculate_file_info(files)
print("file_info",file_info)

extracted_features = []
def myiter(video_file):
    global extracted_features, anomaly_data, file_info
    print(0)
    clip =   VideoFileClip(video_file)
    frames =    clip.iter_frames()
    features =   process_frames_with_anomaly(frames, anomaly_data, file_info[video_file]['file_size'], file_info[video_file]['duration'])
    extracted_features.append(features)
tasks = [Thread(target=myiter,args=(pth,)) for pth in files]
for t in tasks:
    print(t)
    t.start()
for t in tasks:
    print("joining")
    t.join()



file_info {'../Train/anomaly\\3.mp4': {'file_size': [800, 450], 'duration': 140.76}, '../Train/anomaly\\4.mp4': {'file_size': [800, 600], 'duration': 202.52}, '../Train/anomaly\\5.mp4': {'file_size': [720, 576], 'duration': 723.56}}
<Thread(Thread-29 (myiter), initial)>
0
<Thread(Thread-30 (myiter), initial)>
0
<Thread(Thread-31 (myiter), initial)>
0
joining
joining
joining


In [13]:
extracted_features

[{'Time': ['3',
   '3',
   '5',
   '5',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '3',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '3',
   '3',
   '3',
   '3',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',
   '4',


In [14]:
import pandas as pd
df=pd.DataFrame(extracted_features[0])

In [17]:
df.describe()

,Frame,Lighting_Changes,Blurriness,Blocking_Camera,Camera_Movement,Duration
count,396.000000,396.000000,396.000000,396.000000,396.000000,396.00
mean,343.603535,3444.444444,54.963425,4.080808,0.001134,140.76
std,251.036639,3435.665775,31.092637,9.500621,0.002639,0.00
min,1.000000,1728.000000,10.307472,0.000000,0.000000,140.76
25%,107.750000,2451.000000,21.109325,0.000000,0.000000,140.76
50%,213.500000,2797.000000,82.242716,0.000000,0.000000,140.76
75%,593.250000,3220.000000,83.267719,3.000000,0.000833,140.76
max,718.000000,28836.000000,99.690405,64.000000,0.017778,140.76


In [19]:
dfs=[pd.DataFrame(v) for v in extracted_features]

In [20]:
for name_mp4,df in zip(files,dfs):
    name=name_mp4.split('\\')[-1]
    dfs.to_csv(f"../csv/anomaly/{name}.csv")

AttributeError: 'list' object has no attribute 'to_csv'